This demo notebook shows how to use the routines in pbe.py to:
- Given $W\in \mathbb{R}^{m\times n}$, apply the polytope bias estimation (PBE) on $K=\mathbb{S},\mathbb{B}$ to obtain a bias $\alpha^\mathbb{K}$ such that $\operatorname{ReLU}(W\bullet -\alpha^\mathbb{K})$ is injective
- Reconstruct $x$ from $y =\operatorname{ReLU}(Wx-\alpha^\mathbb{K})$

In [25]:
import torch
import numpy as np
import cvxpy as cp
from scipy.optimize import linprog
import scipy as sp
import csv
from scipy.spatial import ConvexHull


def norm_row(W):
    """
    takes a weight matrix W and normalizes the rows
    """
    if torch.is_tensor(W):
        W = W.detach().numpy()
    norm = np.linalg.norm(W, axis=1)
    W_norm = W / norm[:, None]
    return W_norm, norm


def is_omnidir(W):
    """
    takes a weight matrix W and checks for omnidirectionality, binary output
    """
    if torch.is_tensor(W):
        W = W.detach().numpy()
    WT = np.transpose(W)
    m = W.shape[0]
    n = W.shape[1]
    if np.linalg.matrix_rank(W) != n:
        return print('The system is not a frame')
    WW = np.concatenate([WT, [np.ones(m)]])
    ones = np.ones(m)
    zeros = np.concatenate([np.zeros(n), [1]])
    res = linprog(ones, A_eq=WW, b_eq=zeros)

    if res['message'] == 'The algorithm terminated successfully and determined that the problem is infeasible.':
        return False
    elif res['message'] == 'Optimization terminated successfully.':
        if np.any(res['x'] < 1e-10) == True:
            print('0 lies at or very very close to the boundary of the polytope.')
            return True
        else:
            return True

def facets(W):
    """
    computes the facets of the normalized row vectors of the matrix W
    use only with low dimensions!!!!
    """
    if torch.is_tensor(W):
        W = W.detach().numpy()  
    hull = ConvexHull(W)
    facets = hull.simplices

    return list(list(facet) for facet in facets)

def alpha_S(F):
    """
    computes alpha^S for one facet F
    """
    if torch.is_tensor(F):
        F = F.detach().numpy()
    m, n = F.shape
    FT = F.T
    sol = []
    for i in range(m):
        f = np.matmul(F[i, :], FT)
        c = cp.Variable(m)
        soc_constraints = [cp.SOC(1, FT @ c)]  # cp.SOC(1, x) --> ||x||_2 <= 1.
        prob = cp.Problem(cp.Minimize(f @ c), soc_constraints + [c >= np.zeros(m)])
        result = prob.solve()
        sol.append(prob.value)
    return min(sol)


def pbe(W, facets, K='sphere', radius=1):
    """
    The Polytope Bias Estimation for approximating the maximal bias on K.

    Input: a weight matrix W, the list of vertex-facet incidences, the data domain K as string ('sphere', 'ball') and a radius
    Output: radius**-1 * alpha^K
    """

    if torch.is_tensor(W):
        W = W.detach().numpy()
    if is_omnidir(W) == False:
        return 'The frame is not omnidirectional'
    W_norm, norm = norm_row(W)

    m, n = W.shape
    alpha_norm = []

    for vert in range(0, m):
        neighbours = np.unique(np.array(facets)[[vert in facet for facet in facets]])
        corr_vec = W_norm[vert, :].dot(W_norm[neighbours, :].T)
        min_corr = np.min(corr_vec)
        if min_corr < 0:
            min_corr = alpha_S(W_norm[neighbours])
        if K == 'sphere':
            alpha_norm.append(min_corr)
        elif K == 'ball':
            alpha_norm.append(np.min([min_corr,0]))
        else:
            return 'Only sphere and ball are supported as data domains'

    return np.multiply(alpha_norm, np.reciprocal(norm.T)) * radius ** (-1)


def relu(x, W, b):
    """
    computes the forward pass of a ReLU-layer (convention here: negative bias)
    """
    z = np.dot(W, x) - b
    return z * (z > 0)


def relu_inv(z, W, b, facets, mode='facet'):
    """
    reconstructs x from z = ReLU(Wx - b) using a facet-specific left-inverse
    setting mode to something else will use the whole active sub-frame
    """
    I = np.where(z > 0)[0]
    if mode == 'facet':
        for i in range(0, len(facets)):
            if all(k in I for k in facets[i]):
                break
        f_ind = facets[i]
        print('Facet', i, 'with vertices', f_ind, 'is used for reconstruction.')
    else:
        f_ind = I
    W_f = W[f_ind,:]
    b_f = b[f_ind]
    z_f = z[f_ind]
    x = np.linalg.lstsq(W_f, z_f + b_f, rcond=None)[0] # equivalent to synthesis with the canonical dual frame
    return x


def fb_ana(w, a=1):
    '''
    This function returns the frame analysis matrix associated to a collection of filters with decimation factor a.

    Usage:
            W = fb_ana(w, a)
    Output:
            The JN/a x N frame analysis matrix associated with w and decimation factor a.
    '''

    N = w.shape[1]
    J = w.shape[0]
    assert N % a == 0, "a must be a divisor of N"
    W = [np.vstack(sp.linalg.circulant(w[j, :]).T[::a]) for j in range(J)]
    return np.array(W).reshape(J * N // a, N)


def randn_fb(N, J, T=None, scale=True, norm=False, analysis=True, a=1):
    '''
    This function creates a random filterbank with J filters of support T, sampled form a normal distribution and padded with zeros to have length N.
    If scale is set to True, the filters are divided by sqrt(J*T).
    If norm is set to True, the filters are normalized.
    If analysis is set to True, the function returns the frame analysis matrix of the filterbank.
    If analysis is set to False, the function returns the filterbank itself.
    The decimation factor a determined the stride in the convolution and must be a divisor of N.

    Usage:
            W = random_filterbank(N, J)
    Output:
            The NJxN analysis matrix associated with the filterbank
    '''

    assert N % a == 0, "a must be a divisor of N"

    if T == None:
        T = N
    if scale:
        w = np.random.randn(J, T) / np.sqrt(T * J)
    if norm:
        norm = np.linalg.norm(w, axis=1)
        w = w / norm[:, None]
    else:
        w = np.random.randn(J, T)
    w_pad = np.pad(w, ((0, 0), (0, N - T)), constant_values=0)
    if analysis:
        return fb_ana(w_pad, a=a)

    return w_pad

In [26]:
def zcosz(x, W, b):
    """
    computes the forward pass of a zcosz-layer (convention here: negative bias)
    """
    z = np.dot(W, x) - b
    return z * np.cos(z)

def zcosz_inv(z, W, b, facets, mode='facet'):
    """
    reconstructs x from z = zcosz(Wx - b) using a facet-specific left-inverse
    setting mode to something else will use the whole active sub-frame
    """
    I = np.where(z != 0)[0]  # Consider where z is not zero
    if mode == 'facet':
        for i in range(0, len(facets)):
            if all(k in I for k in facets[i]):
                break
        f_ind = facets[i]
        print('Facet', i, 'with vertices', f_ind, 'is used for reconstruction.')
    else:
        f_ind = I
    W_f = W[f_ind,:]
    b_f = b[f_ind]
    z_f = z[f_ind]
    x = np.linalg.lstsq(W_f, z_f * np.cos(z_f) + b_f, rcond=None)[0] # equivalent to synthesis with the canonical dual frame
    return x

In [27]:

import numpy as np


Examples: Icosahedron and random frame on the sphere

In [28]:
phi = (1+np.sqrt(5))/2
ico = np.array([[0,1,phi],[0,1,-phi],[0,-1,phi],[0,-1,-phi],
                [1,phi,0],[-1,phi,0],[1,-phi,0],[-1,-phi,0],
                [phi,0,1],[phi,0,-1],[-phi,0,1],[-phi,0,-1]])/(np.sqrt(1+phi**2))
ran = norm_row(np.random.randn(12,3))[0]

### tetrahedron and mercedes benz frame

In [29]:
tetr = np.array([
    [1, 1, 1],
    [-1, -1, 1],
    [-1, 1, -1],
    [1, -1, -1]
])
mer =  np.array([
    [1, 1, 1, 1],
    [-1, -1, 1, 1],
    [-1, 1, -1, 1],
    [-1, 1, 1, -1],
    [1, -1, -1, 1],
    [1, -1, 1, -1],
])


Compute the vertex-facet incidences

In [30]:
ico_facets = facets(ico)
ran_facets = facets(ran)
tetr_facets=facets(tetr)
mer_facets = facets(mer)
# print(ico_facets)
print(tetr_facets)
print(mer_facets)


[[2, 0, 1], [3, 0, 1], [3, 2, 1], [3, 2, 0]]
[[3, 2, 0, 1], [4, 2, 0, 1], [5, 3, 0, 1], [5, 4, 0, 1], [5, 3, 2, 1], [5, 4, 2, 1], [5, 3, 2, 0], [5, 4, 2, 0]]


PBE on $\mathbb{B}$: get the upper bias $\alpha^\mathbb{B}$

In [31]:
alpha_icosphere = pbe(ico, ico_facets, K='sphere', radius=1)
print('alpha^S for the Icosahedron frame:', alpha_icosphere)
alpha_ico = pbe(ico, ico_facets, K='ball', radius=1)
print('alpha^B for the Icosahedron frame:', alpha_ico)
alpha_ransphere = pbe(ran, ran_facets, K='sphere', radius=1)
print('alpha^S for a random frame:', alpha_ransphere)
alpha_ran = pbe(ran, ran_facets, K='ball', radius=1)
print('alpha^B for a random frame:', alpha_ran)

alpha^S for the Icosahedron frame: [0.4472136 0.4472136 0.4472136 0.4472136 0.4472136 0.4472136 0.4472136
 0.4472136 0.4472136 0.4472136 0.4472136 0.4472136]
alpha^B for the Icosahedron frame: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
alpha^S for a random frame: [ 0.24806507  0.24806507  0.35259811  0.24638486  0.35259811 -0.93397222
  0.46086068 -0.93397222 -0.93397222  0.75681802  0.21958016  0.75214379]


alpha^B for a random frame: [ 0.          0.          0.          0.          0.         -0.93397222
  0.         -0.93397222 -0.93397222  0.          0.          0.        ]


### From theorem 4.4 , alpha_X>=0 , so alpha_B comes out to be zero

In [32]:
alpha_ico3 = pbe(tetr, tetr_facets, K='ball', radius=1)
alpha_ran3 = pbe(mer, mer_facets, K='ball', radius=1)
print('alpha^B for the tetrahedron frame:', alpha_ico3)
print('alpha^B for mercedes-benz frame in R4:', alpha_ran3)

alpha^B for the tetrahedron frame: [-0.57735027 -0.57735027 -0.57735027 -0.57735027]
alpha^B for mercedes-benz frame in R4: [ 0.   0.  -0.5  0.   0.  -0.5]


Reconstruction of a random vector

In [33]:
x = np.random.randn(3)
x = x/np.linalg.norm(x)
z = relu(x, ran, alpha_ran)
x_hat = relu_inv(z, ran, alpha_ran, ran_facets)
print('Error for random polytope:', np.linalg.norm(x-x_hat))

Facet 0 with vertices [2, 3, 5] is used for reconstruction.
Error for random polytope: 4.710277376051325e-16


In [34]:
z1 = relu(x, ico, alpha_ico)
x_hat1 = relu_inv(z1,ico, alpha_ico, ico_facets)
print('Error for icosahedron:', np.linalg.norm(x-x_hat1))

Facet 0 with vertices [5, 0, 10] is used for reconstruction.
Error for icosahedron: 3.510833468576701e-16


In [35]:
z2 = relu(x, tetr, alpha_ico3)
x_hat2 = relu_inv(z2,tetr, alpha_ico3, tetr_facets)
print('Error for tetrahedron:', np.linalg.norm(x-x_hat2))

Facet 0 with vertices [2, 0, 1] is used for reconstruction.
Error for tetrahedron: 3.3306690738754696e-16


# zcosz activation function

### relu activation functions generally has more stable gradient as compared to zcosz, especially for a wider range of inputs.

In [36]:
x = np.random.randn(3)
x = x/np.linalg.norm(x)

z = zcosz(x, ran, alpha_ran)
x_hat = zcosz_inv(z, ran, alpha_ran, ran_facets)
print('Error for random polytope:', np.linalg.norm(x-x_hat))
z1 = zcosz(x, ico, alpha_ico)
x_hat1 = zcosz_inv(z1,ico, alpha_ico, ico_facets)
print('Error for icosahedron:', np.linalg.norm(x-x_hat1))
z2 = zcosz(x, tetr, alpha_ico3)
x_hat2 = zcosz_inv(z2,tetr, alpha_ico3, tetr_facets)
print('Error for tetrahedron:', np.linalg.norm(x-x_hat2))
# z3 = zcosz(x, mer, alpha_ran3)
# x_hat3 = zcosz_inv(z2,mer, alpha_ran3, mer_facets)
# print('Error for tetrahedron:', np.linalg.norm(x-x_hat3))

Facet 0 with vertices [2, 3, 5] is used for reconstruction.
Error for random polytope: 0.010473104331676463
Facet 0 with vertices [5, 0, 10] is used for reconstruction.
Error for icosahedron: 0.39372505759086157
Facet 0 with vertices [2, 0, 1] is used for reconstruction.
Error for tetrahedron: 0.8910252368837295


### Impact on error: For larger values of z, the multiplication by z amplifies the cosine function, leading to a larger difference between zcosz and the regular cosine function (cos(z)). This difference is captured in the absolute error calculation (np.abs(z - np.cos(z))).